# Load Dependencies

In [1]:
from Script.utils import *

/home/hx-gpu3/anaconda3/envs/feng/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

In [2]:
data_dir = './Data/data_cn/'
rundata_dir = './Data/rundata_cn/'
df=pd.read_csv(data_dir+"experiment_index.csv", index_col=0)
df = df.sort_values(by='yield')
df.reset_index(drop=True, inplace=True)
df['entry'] = df.index
df.set_index('entry', inplace=True)
ar_ha_smi=df['Aryl_halide_SMILES'].tolist()
add_smi=df['Additive_SMILES'].tolist()
base_smi=df['Base_SMILES'].tolist()
ligand_smi=df['Ligand_SMILES'].tolist()
yield_value=np.array(df['yield'].tolist())

In [3]:
target = 'yield'
defined_chemical_space = {'Aryl_halide_SMILES':sorted(list(set(ar_ha_smi))), 
                          'Additive_SMILES':sorted(list(set(add_smi))), 
                          'Base_SMILES':sorted(list(set(base_smi))),
                          'Ligand_SMILES':sorted(list(set(ligand_smi))), 
                          }

domain=df.drop(['yield'],axis=1)
print('The size of chemical space is: %d'%len(domain))

The size of chemical space is: 3955


# Reaction Optimization

In [4]:
random_state = 0
des_name = 'alldes' #'ohe','mordred','dft','alldes'
model = 'automl'#'automl','gridsearch','rf'
cc_rate1 = 0.5 # 0-1 exploration proportion of the first Stage
cc_rate2 = 0.2 # 0-1 exploration proportion of the second Stage
task = f'ourwork-{int(cc_rate1*100)}-{int(cc_rate2*100)}-seed' + str(random_state)

In [5]:
desc_map={}
if des_name not in ['ohe','mordred','dft','alldes']:
    print('Error: The selected descriptor is not supported.')
elif des_name == 'alldes':
    desc_map = np.load('./Data/data_cn/all_desc_map.npy',allow_pickle=True).item()
else:
    for i in glob.glob(data_dir+'*.csv'):
        if i.split('/')[-1].split('.')[0].split('_')[-1]==des_name:     
            tem_data=np.array(pd.read_csv(i))
            for j in range(len(tem_data)):
                if des_name in ['ohe','mordred']:
                    desc_map[tem_data[j][0]]=tem_data[j][1:]
                elif des_name=='dft':
                    desc_map[tem_data[j][1]]=tem_data[j][2:]                                   
desc_domain = getdescdomain(domain,desc_map,defined_chemical_space)

In [ ]:
for seed in range(100):
    if os.path.exists(rundata_dir+f'{task}_{des_name}_{model}_{seed}.npy'):   
        continue  
    else:
        yield_optimization_single_line(seed=seed, domain = domain, desc_domain = desc_domain, \
            model = model,des_name = des_name,random_state=random_state,tem_cc1num=(3955*cc_rate1),\
                tem_cc2num=int(3955*cc_rate2),task = task)    
total_results = get_total_results(rundata_dir+f'results_optimization/{task}_{des_name}_{model}_*.npy',start=0,end=100) 
